In [27]:
#------------ Importando las librerias------------
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
#------------ Estableciendo parámetros------------
# Número de rondas 
n_rondas = 100

# Pagos
socios = 3
aprovechado = 5
traidores = 1
tonto = 0

# Extras
ruido = 0.05 # probabilidad de voltear acción elegida. Ingresar valores entre [0, 1]
semilla = 68468 # para funciones con aleatoriedad

# Acciones
C = "C" # cooperar
T = "T" # traicionar

# Matriz de pagos
PAGOS = {
        (C, C): (socios, socios),
        (C, T): (tonto, aprovechado),
        (T, C): (aprovechado, tonto),
        (T, T): (traidores, traidores)}

In [29]:
def pago_ronda(a_i, a_j, pagos = PAGOS):
    """Devuelve (pago_i, pago_j) según las acciones de la ronda."""

    # Por ejemplo:
    # pago_ronda("C", "C") = (3, 3)
    # pago_ronda("C", "T") = (0, 5)

    return pagos[(a_i, a_j)]

In [30]:
### ------------ Estableciendo las clases, padres y hijos ------------
# Clase padres
class Estrategia:
    nombre = "base"
    def reiniciar(self): # Reinicia el "tablero" del juego
        return None

    def decidir(self, mi_historial, historial_rival):
        raise NotImplementedError("Implementa decidir() en la subclase.")

# Clase hijos: 10 estrategias, 10 jugadores:
class OjoPorOjo(Estrategia):
    nombre = "Ojo por ojo"
    def decidir(self, mi_historial, historial_rival):
        # Primera ronda coopera, luego copia la última decisión del rival
        if len(historial_rival) == 0:
            return C
        else:
            return historial_rival[-1]
        
class OjoPorDosOjos(Estrategia):
    nombre = "Ojo por dos ojos"
    # Solo traiciona si el rival traicionó dos veces seguidas (da espacio a que no hayan represalias ante errores de comunicación: ruido, malinterpretaciones )
    def decidir(self, mi_historial, historial_rival):
        # Primera ronda coopera
        if len(historial_rival) < 2:
            return C
        if (historial_rival[-1] == T) and (historial_rival[-2] == T):
            return T
        return C

class SiempreCooperar(Estrategia):
    nombre = "Siempre cooperar"
    def decidir(self, mi_historial, historial_rival):
        return C

class SiempreTraicionar(Estrategia):
    nombre = "Siempre traicionar"
    def decidir(self, mi_historial, historial_rival):
        return T
    
class Aleatorio(Estrategia):
    nombre = "Aleatorio"
    def decidir(self, mi_historial, historial_rival):
        r = random.random()
        if r < 0.50:
            return C
        else:
            return T
        
class Vengativo(Estrategia):
    nombre = "Vengativo" # Si lo traicionan una vez, traicionará para siempre
    def decidir(self, mi_historial, historial_rival):
        if len(historial_rival) == 0:
            return C
        conteo_traiciones = 0
        for accion in historial_rival:
            if accion == T:
                conteo_traiciones = conteo_traiciones + 1
        if conteo_traiciones > 0:
            return T
        else:
            return C
        
class Pavlov(Estrategia):
    nombre = "Pavlov (Gana-Permanece)"
    # Repetir la acción anterior si el pago fue aprovechado = 5 o socios = 3
    # Cambiar la acción anterior si el pago fue traidores = 1 o tonto = 0
    def decidir(self, mi_historial, historial_rival):
        if len(mi_historial) == 0:
            return C
        accion_mia_prev = mi_historial[-1]
        accion_rival_prev = historial_rival[-1]
        # Cambiar acción
        if (accion_mia_prev == T) and (accion_rival_prev == T):
            return C
        if (accion_mia_prev == C) and (accion_rival_prev == T):
            return T
        # Mantener acción
        if (accion_mia_prev == C) and (accion_rival_prev == C):
            return C
        if (accion_mia_prev == T) and (accion_rival_prev == C):
            return T
        
class Alternadora(Estrategia):
    nombre = "Alternadora"
    def decidir(self, mi_historial, historial_rival): # Alterna cooperar y traicionar
        return C if len(mi_historial) % 2 == 0 else T
        
class Joss(Estrategia):
    nombre = "Joss" # Ojo por ojo con tentación de traicionar
    def __init__(self, p=0.10): # probabilidad de traicionar de 10.0%
        self.p = p  # probabilidad de traicionar aleatoriamente cuando tocaría cooperar

    def decidir(self, mi_historial, historial_rival):
        # Actúa como Ojo por ojo
        if len(historial_rival) == 0:
            return C
        accion = historial_rival[-1]
        # Cambia a traición con prob p aunque debiera cooperar según ojo por ojo
        if accion == C:
            if random.random() < self.p:
                return T
        return accion
    
class Umbral60(Estrategia):
    nombre = "Umbral 60%"
    def decidir(self, mi_historial, historial_rival): # Si el rival ha cooperado en más de 60% de las jugadas, coopera, si no traiciona
        if len(historial_rival) == 0:
            return C # Empieza cooperando
        cooperaciones = historial_rival.count(C)
        p = cooperaciones / len(historial_rival)
        return C if p > 0.60 else T
    
class TraidorCada7(Estrategia):
    nombre = "Traiciona cada 7" # Cada siete turnos traiciona, de lo contrario juega Ojo por Ojo
    def decidir(self, mi_historial, historial_rival):
        turno = len(mi_historial) + 1
        if turno % 7 == 0:
            return T
        return historial_rival[-1]



In [31]:
# Jugadas con opción de ruido de comunicación

def aplicar_ruido(accion, prob:float, rng):
    # Con una probabilidad elegida (5.0%, por ejemplo), el jugador escoge la acción contraria a la que normalmente su estrategia le dicta.
    # Objetivo: simular mejor el mundo real en que, por problemas de comunicación, una acción se puede malinterpretar. Analizar cómo se
    # comportan estrategias en ambientes con imperfección.
    
    prob = float(prob)
    if prob == 0.0:
        return accion

    # Voltea decisión con probabilidad p
    return (C if accion == T else T) if rng.random() < prob else accion

In [42]:

def jugar_partida(j1: Estrategia, j2: Estrategia, n=n_rondas, prob_ruido=ruido):
   
    # Reiniciar estado interno
    j1.reiniciar()
    j2.reiniciar()

    rng = random.Random(semilla)

    # Historiales
    hist1 = []
    hist2 = []
    # Listas de acciones y pagos
    # acciones_lista = [] ---------------
    pagos_lista = []

    suma1 = 0
    suma2 = 0

    for i in range(n):
        a1 = j1.decidir(hist1, hist2) # Cada jugador decide usando su propio historial y el del rival
        a2 = j2.decidir(hist2, hist1)

        a1 = aplicar_ruido(a1, prob_ruido, rng) # Aplicación de ruido
        a2 = aplicar_ruido(a2, prob_ruido, rng)

        # acciones_lista.append((a1, a2))

        (p1, p2) = PAGOS[(a1, a2)]
        pagos_lista.append((p1, p2))

        # Actualización de historiales de acciones observadas
        hist1.append(a1)
        hist2.append(a2)

        suma1 = suma1 + p1
        suma2 = suma2 + p2

    # Promedio de puntos para cada jugador al final del partido
    
    prom1 = suma1 / n # Promedio obtenido
    prom2 = suma2 / n


    return {
        "pagos": pagos_lista,
        "promedios": (prom1, prom2),
        "hist1": hist1,
        "hist2": hist2,
    }






In [43]:
j1 = OjoPorOjo()
j2 = SiempreCooperar()
j3 = SiempreTraicionar()

jugadores = {1: j1, 2: j2, 3: j3}
nombres_jugadores = {1: j1.nombre, 2: j2.nombre, 3: j3.nombre}

for i in range(1,3+1):
    for j in range(1,3+1):
        if i != j:
            nombre = str(f"res_{i}{j}")
            nombre =jugar_partida(jugadores[i], jugadores[j], n=100, prob_ruido=0)
            print(f"J{i} ({nombres_jugadores[i]}) vs J{j} ({nombres_jugadores[j]}):", nombre["promedios"])
            # print(f"Primeras 3 acciones:", nombre["acciones"][:3])
            print(f"Primeros 3 pagos:", nombre["pagos"][:3])
            print()

J1 (Ojo por ojo) vs J2 (Siempre cooperar): (3.0, 3.0)
Primeros 3 pagos: [(3, 3), (3, 3), (3, 3)]

J1 (Ojo por ojo) vs J3 (Siempre traicionar): (0.99, 1.04)
Primeros 3 pagos: [(0, 5), (1, 1), (1, 1)]

J2 (Siempre cooperar) vs J1 (Ojo por ojo): (3.0, 3.0)
Primeros 3 pagos: [(3, 3), (3, 3), (3, 3)]

J2 (Siempre cooperar) vs J3 (Siempre traicionar): (0.0, 5.0)
Primeros 3 pagos: [(0, 5), (0, 5), (0, 5)]

J3 (Siempre traicionar) vs J1 (Ojo por ojo): (1.04, 0.99)
Primeros 3 pagos: [(5, 0), (1, 1), (1, 1)]

J3 (Siempre traicionar) vs J2 (Siempre cooperar): (5.0, 0.0)
Primeros 3 pagos: [(5, 0), (5, 0), (5, 0)]

